In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load the dataset
data = pd.read_csv("dataset.csv")

# Extract features and labels
image_paths = data["image_path"]
labels = data["artist_label"]

# Function to load and preprocess images while preserving aspect ratio
def load_and_preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path)  # Load image without resizing
        img = image.img_to_array(img)
        
        # Calculate target width and height while preserving aspect ratio
        height, width, _ = img.shape
        target_height, target_width = target_size
        
        # Resize image with aspect ratio preserved
        if width > height:
            new_width = target_width
            new_height = int((height / width) * target_width)
        else:
            new_height = target_height
            new_width = int((width / height) * target_height)
        
        img_resized = image.array_to_img(img)
        img_resized = img_resized.resize((new_width, new_height))

        # Pad the image to target size
        img_padded = np.zeros((target_height, target_width, 3))  # Create a black canvas
        img_padded[:new_height, :new_width, :] = np.array(img_resized)  # Place the resized image on the canvas
        
        images.append(img_padded)
    
    return np.array(images)

# Load and preprocess images
X = load_and_preprocess_images(image_paths)
X = X / 255.0  # Normalize the images to [0, 1] range

# Convert labels to numpy array and one-hot encode them
y = np.array(labels)
y = to_categorical(y, num_classes=3)  # 3 classes (0, 1, 2)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a Convolutional Neural Network (CNN) model for multi-class classification
model = tf.keras.Sequential([
    # First convolutional block
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Second convolutional block
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Third convolutional block
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Flatten the output
    tf.keras.layers.Flatten(),
    
    # Fully connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Output layer with 3 units for multi-class classification
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes, softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Categorical cross-entropy for multi-class classification
              metrics=['accuracy'])

# Data augmentation to improve generalization
datagen = ImageDataGenerator(
    rotation_range=0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the data augmentation generator
datagen.fit(X_train)

# Train the model with augmented data
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

# Predict on the test set
y_pred = np.argmax(model.predict(X_test), axis=1)  # Convert softmax probabilities to class labels

# Get the actual class labels (as integers)
y_true = np.argmax(y_test, axis=1)

# Classification report for performance evaluation
report = classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1', 'Class 2'])
print(report)


c:\Users\Chien Shyong\Documents\School Work\Term 8\Data Science\yukien\myvenv\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
c:\Users\Chien Shyong\Documents\School Work\Term 8\Data Science\yukien\myvenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Chien Shyong\Documents\School Work\Term 8\Data Science\yukien\myvenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`,

Epoch 1/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 23s 322ms/step - accuracy: 0.5358 - loss: 0.9581 - val_accuracy: 0.7700 - val_loss: 0.5762
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 21s 349ms/step - accuracy: 0.6920 - loss: 0.6985 - val_accuracy: 0.8070 - val_loss: 0.5312
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 20s 330ms/step - accuracy: 0.7492 - loss: 0.6149 - val_accuracy: 0.8357 - val_loss: 0.4586
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 289ms/step - accuracy: 0.7736 - loss: 0.5593 - val_accuracy: 0.8296 - val_loss: 0.4565
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 285ms/step - accuracy: 0.7940 - loss: 0.5440 - val_accuracy: 0.8727 - val_loss: 0.3996
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 287ms/step - accuracy: 0.8151 - loss: 0.4654 - val_accuracy: 0.8275 - val_loss: 0.4438
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 286ms/step - accuracy: 0.7988 - loss: 0.4995 - val_accuracy: 0.8727 - val_loss: 0.3582
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 286ms/step - accuracy: 0.8257 - loss: 0.4680 - val_accu